# **Instalação dos Pacotes**

In [ ]:
!pip install playwright

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 26.1 MB/s eta 0:00:00


In [ ]:
!playwright install

152.8 MiB [] 0% 10.1s152.8 MiB [] 0% 35.0s152.8 MiB [] 0% 15.4s152.8 MiB [] 0% 10.9s152.8 MiB [] 0% 10.0s152.8 MiB [] 1% 7.0s152.8 MiB [] 1% 6.0s152.8 MiB [] 2% 5.7s152.8 MiB [] 2% 5.4s152.8 MiB [] 2% 5.6s152.8 MiB [] 3% 5.5s152.8 MiB [] 3% 5.7s152.8 MiB [] 3% 5.3s152.8 MiB [] 4% 5.2s152.8 MiB [] 4% 5.4s152.8 MiB [] 4% 5.6s152.8 MiB [] 4% 5.4s152.8 MiB [] 5% 5.3s152.8 MiB [] 5% 5.1s152.8 MiB [] 6% 4.9s152.8 MiB [] 7% 4.6s152.8 MiB [] 8% 4.4s152.8 MiB [] 8% 4.3s152.8 MiB [] 9% 4.2s152.8 MiB [] 10% 4.1s152.8 MiB [] 11% 4.0s152.8 MiB [] 12% 4.0s152.8 MiB [] 12% 4.1s152.8 MiB [] 13% 4.2s152.8 MiB [] 13% 4.1s152.8 MiB [] 13% 4.2s152.8 MiB [] 14% 4.1s152.8 MiB [] 15% 4.1s152.8 MiB [] 16% 4.0s152.8 MiB [] 17% 3.9s152.8 MiB [] 18% 3.8s152.8 MiB [] 19% 3.6s152.8 MiB [] 20% 3.5s152.8 MiB [] 20% 3.4s152.8 MiB [] 21% 3.3s152.8 MiB [] 22% 3.3s152.8 MiB [] 22% 3.2s152.8 MiB [] 23% 3.2s152.8 MiB [] 23% 3.1s152.8 MiB [] 24% 3.0s152.8 MiB [] 25% 3.0s152.8 MiB [] 26% 2.9s152.8 MiB [] 27% 2.8s152.8 MiB [

# **Bibliotecas**

In [ ]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
import re
import asyncio
import requests
from google.colab import drive
from urllib.parse import urlparse

# **Funções utilizadas para fazer o scraping dos dados**

Captura HTML

In [ ]:
async def get_html(url, selector, sleep=5, retries=3):
  html = None
  for i in range(1, retries+1):
    #await asyncio.sleep(sleep * i)

    try:
      async with async_playwright() as p:
        browser = await p.firefox.launch() #chromium
        page = await browser.new_page()
        await page.goto(url)
        #print(await page.title())
        html = await page.inner_html(selector)
    except PlaywrightTimeout:
      print(f'Timeout em {url}')
    else:
      break

  return html

Para cada cidade recupera o total de páginas de anúncios

In [ ]:
async def extract_total_pages(html):
    soup = BeautifulSoup(html, 'html.parser')

    links = soup.find_all('a')
    if links:
        last_link = links[-1].get('href')
        if last_link:
            page_param = last_link.split('=')[-1] # Verificar se o último link contém o parâmetro 'pagina'
            if page_param.isdigit():
                total_pages = int(page_param)
                return total_pages
    return 1 # Se não houver links ou o último link não contiver um número válido, assumir que há apenas uma página

Função que retorna um array com todas as urls do site


In [ ]:
async def scrape_links(html, url):
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a')
    href = [l['href'] for l in links]
    standings_pages = [url + l for l in href]
    all_pages_urls = []

    for url0 in standings_pages:
        html = await get_html(url0, '#__next .MuiPagination-ul')
        #Todas as páginas de uma cidade
        total_pages = await extract_total_pages(html) # Recupera o total de páginas de cada cidade

        all_pages_urls.extend([f"{url0}?pagina={page}" for page in range(1, total_pages + 1)]) # Monta as urls que contém as informações dos imóveis

    return all_pages_urls


Captura os dados dos imóveis de cada localidade

In [ ]:
async def extrair_dados_imoveis(url):
    # Parseia a URL para obter o estado e a cidade
    parsed_url = urlparse(url)
    path_components = parsed_url.path.split('/')
    estado = path_components[3].upper()  # O estado está na posição 3
    cidade = path_components[4].replace('-', ' ').title()  # A cidade está na posição 4 e pode conter hífens

    # HTML da página
    response = requests.get(url)
    html = response.text
    #html = await get_html(url, '#__next')
    soup = BeautifulSoup(html, 'html.parser')

    # Encontra todos os elementos com a classe "jss152"
    imoveis = soup.find_all(class_="jss152")

    # Inicializa listas para armazenar valores e localizações
    valores = []
    localizacoes = []

    # Itera sobre os elementos encontrados
    for imovel in imoveis:
        if imovel.text.startswith('R$'):
            valores.append(imovel.text[3:])
        elif imovel.name == 'h2':
            # Extrai somente o bairro da localização
            localizacao_termos = imovel.text.split(', ')
            bairro = localizacao_termos[-1]
            localizacoes.append(bairro)

    # Retorna uma lista contendo os valores formatados
    return [(estado, cidade, valor, localizacao) for valor, localizacao in zip(valores, localizacoes)]


Função Principal

In [ ]:
async def main():

    url = 'https://loft.com.br'
    # Seleciona todos os links das cidades
    cityselector = '#__next .jss25'
    html = await get_html(url, cityselector)

    drive.mount('/content/drive')
    all_links = await scrape_links(html, url)

    arquivo = '/content/drive/MyDrive/resultado_imoveis.txt'
    dados_imoveis = []

    tentativas = 3
    for url in all_links:
        for j in range(tentativas):
            try:
                print(url)
                dados_imoveis.extend(await extrair_dados_imoveis(url))
                break  # Se a operação for bem-sucedida, sai do loop
            except Exception as e:
                if j + 1 < tentativas:
                    print(f"Tentativa {j+1} de {tentativas}: Erro ao extrair dados da URL {url}. Tentando novamente em 5 segundos...")
                    await asyncio.sleep(5)  # Espera 5 segundos antes de tentar novamente
                else:
                    print(f"Erro após {tentativas} tentativas ao extrair dados da URL {url}: {e}")

    # Escreve os resultados em um arquivo
    with open(arquivo, 'w') as f:
        f.write(f'uf;cidade;valor;bairro\n')
        for dados in dados_imoveis:
            f.write(f'{dados[0]};{dados[1]};{dados[2]};{dados[3]}\n')

    print("Arquivo criado com sucesso!")


# Execução da função main

*É realizada a busca de apartamentos em todas as cidades. Este procedimento leva em torno de 15 minutos*


In [ ]:
await main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=1
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=2
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=3
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=4
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=5
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=6
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=7
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=8
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=9
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=10
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=11
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=12
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=13
https://loft.com.br/venda/apartamentos/sp/sao-paulo?pagina=14
https://loft

# **PySpark**
Instalação do pacotes

In [ ]:
!pip install -q findspark
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=81661679189cc54bb25e738b8076ea9dae6980132fbb94897c658fb2c0e6d9a2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [ ]:
import os
import sys
import pyspark

In [ ]:
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Primeiro Exemplo Spark") \
       .getOrCreate()

spark

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/resultado_imoveis.txt", header=True, sep=';', inferSchema=True)

In [ ]:
df.count()

38913

In [ ]:
df.show()

+---+---------+---------+-----------------+
| uf|   cidade|    valor|           bairro|
+---+---------+---------+-----------------+
| SP|Sao Paulo|  827.999|             Pari|
| SP|Sao Paulo|2.300.000|          Cursino|
| SP|Sao Paulo|1.810.000| Jardim São Paulo|
| SP|Sao Paulo|  439.900|         Tremembé|
| SP|Sao Paulo|  850.000|         Tucuruvi|
| SP|Sao Paulo|3.000.000|Jardim Paulistano|
| SP|Sao Paulo|  800.000|         Mandaqui|
| SP|Sao Paulo|1.100.000|         Brooklin|
| SP|Sao Paulo|1.880.000|     Moema Índios|
| SP|Sao Paulo|  425.000| Jardim São Paulo|
| SP|Sao Paulo|  217.500|          Jaraguá|
| SP|Sao Paulo|  347.112|       Ponte Rasa|
| SP|Sao Paulo|1.100.000|    Santa Cecília|
| SP|Sao Paulo|  520.000|        Liberdade|
| SP|Sao Paulo|  205.000|        Sapopemba|
| SP|Sao Paulo|  500.000|         Mandaqui|
| SP|Sao Paulo|  298.000|     Cidade Dutra|
| SP|Sao Paulo|2.400.000|       Bela Vista|
| SP|Sao Paulo|  336.000|         Mandaqui|
| SP|Sao Paulo|  285.000|       

In [ ]:
import pyspark.sql.functions as func

In [ ]:
df = df.withColumn("valor", func.regexp_replace("valor", "\.", ""))

In [ ]:
df.show()

+---+---------+-------+-----------------+
| uf|   cidade|  valor|           bairro|
+---+---------+-------+-----------------+
| SP|Sao Paulo| 827999|             Pari|
| SP|Sao Paulo|2300000|          Cursino|
| SP|Sao Paulo|1810000| Jardim São Paulo|
| SP|Sao Paulo| 439900|         Tremembé|
| SP|Sao Paulo| 850000|         Tucuruvi|
| SP|Sao Paulo|3000000|Jardim Paulistano|
| SP|Sao Paulo| 800000|         Mandaqui|
| SP|Sao Paulo|1100000|         Brooklin|
| SP|Sao Paulo|1880000|     Moema Índios|
| SP|Sao Paulo| 425000| Jardim São Paulo|
| SP|Sao Paulo| 217500|          Jaraguá|
| SP|Sao Paulo| 347112|       Ponte Rasa|
| SP|Sao Paulo|1100000|    Santa Cecília|
| SP|Sao Paulo| 520000|        Liberdade|
| SP|Sao Paulo| 205000|        Sapopemba|
| SP|Sao Paulo| 500000|         Mandaqui|
| SP|Sao Paulo| 298000|     Cidade Dutra|
| SP|Sao Paulo|2400000|       Bela Vista|
| SP|Sao Paulo| 336000|         Mandaqui|
| SP|Sao Paulo| 285000|          Cursino|
+---+---------+-------+-----------

In [ ]:
df = df.withColumn("valor", df["valor"].cast("int"))
df.show()

+---+---------+-------+-----------------+
| uf|   cidade|  valor|           bairro|
+---+---------+-------+-----------------+
| SP|Sao Paulo| 827999|             Pari|
| SP|Sao Paulo|2300000|          Cursino|
| SP|Sao Paulo|1810000| Jardim São Paulo|
| SP|Sao Paulo| 439900|         Tremembé|
| SP|Sao Paulo| 850000|         Tucuruvi|
| SP|Sao Paulo|3000000|Jardim Paulistano|
| SP|Sao Paulo| 800000|         Mandaqui|
| SP|Sao Paulo|1100000|         Brooklin|
| SP|Sao Paulo|1880000|     Moema Índios|
| SP|Sao Paulo| 425000| Jardim São Paulo|
| SP|Sao Paulo| 217500|          Jaraguá|
| SP|Sao Paulo| 347112|       Ponte Rasa|
| SP|Sao Paulo|1100000|    Santa Cecília|
| SP|Sao Paulo| 520000|        Liberdade|
| SP|Sao Paulo| 205000|        Sapopemba|
| SP|Sao Paulo| 500000|         Mandaqui|
| SP|Sao Paulo| 298000|     Cidade Dutra|
| SP|Sao Paulo|2400000|       Bela Vista|
| SP|Sao Paulo| 336000|         Mandaqui|
| SP|Sao Paulo| 285000|          Cursino|
+---+---------+-------+-----------

In [ ]:
df.createOrReplaceTempView("apartamentos_loft")

# *Query que recupera o valor minimo do arquivo*

In [ ]:
min = df.agg(func.min('valor').alias('minimo'))
min = min.withColumn("minimo", func.concat(func.lit("R$ "), func.translate(func.format_number(min["minimo"],2),",.",".,")))

min.show(n=30, truncate=False)

+------------+
|minimo      |
+------------+
|R$ 60.000,00|
+------------+



# *Query que recupera o valor máximo do arquivo*

In [ ]:
max = df.agg(func.max('valor').alias('maximo'))
max = max.withColumn("maximo", func.concat(func.lit("R$ "), func.translate(func.format_number(max["maximo"],2),",.",".,")))

max.show(n=30, truncate=False)

+----------------+
|maximo          |
+----------------+
|R$ 45.000.000,00|
+----------------+



# *Query que recupera o valor minimo, máximo e média dos apartamentos de cada cidade*

In [ ]:
result = spark.sql(
"""
  SELECT
    apartamentos_loft.uf,
    apartamentos_loft.cidade,
    MIN(apartamentos_loft.valor) AS VALOR_MIN,
    MAX(apartamentos_loft.valor) AS VALOR_MAX,
    REPLACE(CAST(AVG(apartamentos_loft.valor) AS FLOAT),'.',',') AS VALOR_AVG
  FROM apartamentos_loft
  GROUP BY apartamentos_loft.uf, apartamentos_loft.cidade
  ORDER BY 1,2;
""")
result.show(n=100)

+---+--------------------+---------+---------+----------+
| uf|              cidade|VALOR_MIN|VALOR_MAX| VALOR_AVG|
+---+--------------------+---------+---------+----------+
| MG|      Belo Horizonte|    63830|  5320000| 572972,06|
| MG|           Nova Lima|   210000|  9926385| 1834789,9|
| PR|            Curitiba|   139000| 11968627| 1259052,0|
| RJ|             Niteroi|    80000|  8600000| 896461,94|
| RJ|      Rio De Janeiro|    73685| 33000000| 1021133,1|
| RS|     Bento Goncalves|   230000|  2200000|  906285,7|
| RS|        Cachoeirinha|   109900|  4500000| 345237,34|
| RS|              Canela|   299000|  2950000|  893854,7|
| RS|              Canoas|    75000|  3500000| 424840,44|
| RS|      Capao Da Canoa|   190000|  6500000|  914159,9|
| RS|       Caxias Do Sul|   122000|  4900000|  657779,7|
| RS|              Esteio|    95000|  4452400|  757432,0|
| RS|             Gramado|    90000| 14793750| 1930752,6|
| RS|            Gravatai|    90000|  2447000| 307872,16|
| RS|         

# *Query que recupera o valor minimo, máximo e média dos apartamentos de cada bairro de cada cidade*

In [ ]:
result = spark.sql(
"""
  SELECT
    apartamentos_loft.uf,
    apartamentos_loft.cidade,
    apartamentos_loft.bairro AS BAIRRO,
    MIN(apartamentos_loft.valor) AS VALOR_MIN,
    MAX(apartamentos_loft.valor) AS VALOR_MAX,
    REPLACE(CAST(AVG(apartamentos_loft.valor) AS FLOAT),'.',',') AS VALOR_AVG
  FROM apartamentos_loft
  GROUP BY apartamentos_loft.uf, apartamentos_loft.cidade, apartamentos_loft.bairro
  ORDER BY 1,2;
""")
result.show(n=100)

+---+--------------+--------------------+---------+---------+----------+
| uf|        cidade|              BAIRRO|VALOR_MIN|VALOR_MAX| VALOR_AVG|
+---+--------------+--------------------+---------+---------+----------+
| MG|Belo Horizonte|           Coqueiros|   213000|   449500|  314000,0|
| MG|Belo Horizonte|Bairro das Indúst...|   450000|   450000|  450000,0|
| MG|Belo Horizonte|        Águas Claras|    69149|   280000|  135146,4|
| MG|Belo Horizonte|        Bandeirantes|   230000|  5899000| 2410105,2|
| MG|Belo Horizonte|           Concórdia|   260000|   650000| 395625,84|
| MG|Belo Horizonte|         Jonas Veiga|   166000|   182000| 176333,33|
| MG|Belo Horizonte|             Cardoso|   170000|   782000| 406160,66|
| MG|Belo Horizonte|               Graça|   335000|  1440000| 614115,56|
| MG|Belo Horizonte|           Dom Bosco|   240000|   395000| 280733,34|
| MG|Belo Horizonte|        Nova Granada|   240000|  1300000| 540452,25|
| MG|Belo Horizonte|          Luxemburgo|   305000|